## Installation

In [1]:
!pip install openai
!pip install plotly
!pip install git+https://github.com/openai/swarm.git
!pip install firecrawl-py

  Cloning https://github.com/openai/swarm.git to /tmp/pip-req-build-lkbui7wh
  Running command git clone --filter=blob:none --quiet https://github.com/openai/swarm.git /tmp/pip-req-build-lkbui7wh
  Resolved https://github.com/openai/swarm.git to commit 9db581cecaacea0d46a933d6453c312b034dbf47
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Adding OpenAI API Key

In [2]:
import openai
import os
from google.colab import userdata

openai.api_key = userdata.get('OPENAI_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_KEY')

## Loading the Dataset

In [16]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/noelabu/InsightCart/refs/heads/main/data/sales_data.csv')

## Data Detective

**Data Detective Agent** is designed to ensure the integrity, accuracy, and security of your data. Comprised of three specialized agents—**Data Cleaning Agent**, **Anomaly Detection Agent**, and **Transaction Risk Agent**—the Data Detective Agent works in tandem to optimize datasets and identify critical issues. Here’s a breakdown of each agent's capabilities:

1. **Data Cleaning Agent**  
   The **Data Cleaning Agent** focuses on the simple yet essential task of cleaning and validating your dataset. It efficiently scans for issues like **missing values**, **duplicate rows**, and **inconsistent data types**. This agent provides tailored **cleaning recommendations** to address these issues, ensuring that your dataset is accurate and ready for analysis. Whether it's suggesting imputation techniques for missing values or identifying columns with invalid data types, the Data Cleaning Agent helps enhance the quality of your data with minimal effort.

2. **Anomaly Detection Agent**  
   The **Anomaly Detection Agent** is designed to identify outliers and abnormal patterns within your dataset. It scans each column for both **low** and **high anomalies**, flagging data points that deviate significantly from expected values. After detecting these outliers, the agent generates a **cleaning recommendation report**, suggesting how to handle these anomalies—whether through imputation, removal, or further investigation. This agent is especially useful for identifying errors or unexpected trends that could skew analysis or indicate deeper issues within the dataset.

3. **Transaction Risk Agent**  
   The **Transaction Risk Agent** analyzes transaction-level data to detect potential fraud, **suspicious activity**, or **financial risks**. It identifies **high-value transactions** by assessing revenue against predefined thresholds (e.g., the 95th percentile), analyzes transaction patterns, and detects anomalies in attributes like **device type** or transaction frequency. The agent generates a **comprehensive risk assessment report** that highlights flagged transactions, provides statistical insights, and assigns a **risk level** to the dataset. By offering **actionable recommendations**—such as manual reviews or enhanced security measures—the Transaction Risk Agent helps organizations proactively address potential fraud, improving the security and integrity of financial transactions.

In [4]:
from swarm import Swarm, Agent
import pandas as pd
import numpy as np
import scipy.stats as stats
import json

class DataDetectiveAgent:
    def __init__(self):
        """
        Initialize the Data Detective Agent
        """
        # Initialize Swarm client
        self.client = Swarm()

        self.cleaning_agent = Agent(
            name="Data Cleaning Agent",
            model="gpt-4o-mini",
            instructions=(
                "You are an expert in data cleaning and preprocessing. "
                "Identify and resolve data quality issues with precision. "
                "Provide detailed, actionable recommendations for data cleaning."
                "Respond ONLY with a JSON format output directly addressing the cleaning recommendations. No json in the start of the string."
            )
        )

        self.anomaly_agent = Agent(
            name="Anomaly Detection Agent",
            model="gpt-4o-mini",
            instructions=(
                "You are a specialist in statistical anomaly detection. "
                "Analyze data for unusual patterns, outliers, and potential anomalies. "
                "Provide comprehensive statistical insights."
                "Respond ONLY with a JSON format output directly addressing the cleaning recommendations. No json in the start of the string."

            )
        )

        self.transaction_agent = Agent(
            name="Transaction Risk Agent",
            model="gpt-4o-mini",
            instructions=(
                "You are an expert in transaction risk assessment. "
                "Analyze transactions for potential fraud or suspicious activity. "
                "Provide detailed risk scoring and recommendations."
                "Respond ONLY with a JSON format output directly addressing the risk assessment. No json in the start of the string."
            )
        )

    def clean_and_validate_data(self, df: pd.DataFrame) -> dict:
        """
        Perform data cleaning using the Swarm cleaning agent

        :param df: Input pandas DataFrame
        :return: Cleaning report and cleaned DataFrame
        """
        # Prepare data issues for analysis
        data_issues = {
            'missing_values': df.isnull().sum().to_dict(),
            'duplicate_rows': df.duplicated().sum(),
            'data_type_issues': df.dtypes.astype(str).to_dict()
        }

        # Run cleaning agent analysis
        cleaning_response = self.client.run(
            agent=self.cleaning_agent,
            messages=[{
                "role": "user",
                "content": f"""Analyze these data issues:
                Missing Values: {str(data_issues['missing_values'])}
                Duplicate Rows: {data_issues['duplicate_rows']}
                Data Type Issues: {str(data_issues['data_type_issues'])}

                Provide a comprehensive cleaning strategy."""
            }]
        )

        # Perform basic cleaning
        cleaned_df = df.copy()
        cleaned_df.dropna(inplace=True)
        cleaned_df.drop_duplicates(inplace=True)

        return {
            'cleaning_report': cleaning_response.messages[-1]['content'],
            'cleaned_dataframe': cleaned_df,
            'data_issues': data_issues
        }

    def detect_anomalies(self, df: pd.DataFrame) -> dict:
        """
        Perform anomaly detection using the Swarm anomaly agent

        :param df: Input pandas DataFrame
        :return: Anomaly detection report
        """
        # Numerical columns for anomaly detection
        numerical_columns = df.select_dtypes(include=[np.number]).columns

        # Statistical summary
        statistical_summary = df[numerical_columns].describe()

        # Anomaly detection using Z-score
        potential_anomalies = {}
        for column in numerical_columns:
            z_scores = np.abs(stats.zscore(df[column]))
            anomalies = df[z_scores > 3]
            potential_anomalies[column] = {
                'anomaly_count': len(anomalies),
                'anomaly_percentage': len(anomalies) / len(df) * 100
            }

        # Run anomaly detection agent
        anomaly_response = self.client.run(
            agent=self.anomaly_agent,
            messages=[{
                "role": "user",
                "content": f"""Analyze these potential anomalies:
                Statistical Summary: {str(statistical_summary)}
                Potential Anomalies: {str(potential_anomalies)}

                Provide a detailed anomaly detection report."""
            }]
        )

        return {
            'anomaly_report': anomaly_response.messages[-1]['content'],
            'potential_anomalies': potential_anomalies,
            'statistical_summary': statistical_summary
        }

    def flag_suspicious_transactions(self, df: pd.DataFrame) -> dict:
        """
        Flag suspicious transactions using the transaction risk agent

        :param df: Input pandas DataFrame
        :return: Transaction flagging report
        """
        sus_df = df.copy(deep=True)

        # Convert 'revenue' column to numeric
        sus_df["revenue"] = sus_df["revenue"].apply(lambda x: int(str(x).replace("₱", "").replace(",", "")))

        # Define suspicious transaction indicators
        suspicious_indicators = {
            'high_value_transactions': sus_df[sus_df['revenue'] > sus_df['revenue'].quantile(0.95)]
        }

        # Transaction summary
        transaction_summary = sus_df[['revenue', 'transactions', 'device_type']].describe()

        # Run transaction risk agent
        transaction_response = self.client.run(
            agent=self.transaction_agent,
            messages=[{
                "role": "user",
                "content": f"""Analyze these transaction details:
                Transaction Summary: {transaction_summary}
                Suspicious Indicators: {suspicious_indicators}

                Provide a comprehensive transaction risk assessment."""
            }]
        )

        return {
            'transaction_report': transaction_response.messages[-1]['content'],
            'suspicious_indicators': suspicious_indicators
        }

    def comprehensive_analysis(self, df: pd.DataFrame) -> dict:
        """
        Perform a comprehensive multi-agent analysis

        :param df: Input pandas DataFrame
        :return: Comprehensive analysis report
        """

        # Perform individual analysis steps
        cleaning_results = self.clean_and_validate_data(df)
        anomaly_results = self.detect_anomalies(cleaning_results['cleaned_dataframe'])
        transaction_results = self.flag_suspicious_transactions(cleaning_results['cleaned_dataframe'])

        # Combine results
        comprehensive_report = {
            'data_cleaning': cleaning_results,
            'anomaly_detection': anomaly_results,
            'transaction_analysis': transaction_results
        }

        return comprehensive_report

In [5]:
# Initialize Data Detective Agent
detective = DataDetectiveAgent()


In [6]:
# Run comprehensive analysis
analysis_results = detective.comprehensive_analysis(df)

In [7]:
cleaning_report = analysis_results["data_cleaning"]["cleaning_report"]
print(cleaning_report)

{
  "data_cleaning_recommendations": {
    "missing_values": {
      "status": "No missing values detected",
      "action_required": "None"
    },
    "duplicate_rows": {
      "status": "No duplicate rows detected",
      "action_required": "None"
    },
    "data_type_issues": {
      "date": {
        "current_type": "object",
        "recommended_type": "datetime64",
        "action": "Convert the 'date' column to datetime format using pd.to_datetime()."
      },
      "source": {
        "current_type": "object",
        "recommended_type": "category",
        "action": "Convert the 'source' column to categorical type for better memory optimization using astype('category')."
      },
      "medium": {
        "current_type": "object",
        "recommended_type": "category",
        "action": "Convert the 'medium' column to categorical type for better memory optimization using astype('category')."
      },
      "delivery_available": {
        "current_type": "object",
        "re

In [8]:
anomaly_report = analysis_results["anomaly_detection"]["anomaly_report"]
print(anomaly_report)

{
  "cleaning_recommendations": {
    "pageviews": {
      "outlier_thresholds": {
        "lower": 0,
        "upper": 34832
      },
      "actions": [
        "Investigate and potentially remove records with exceptionally high pageviews (above 30000).",
        "Check for data entry errors for low pageviews (0) where not applicable."
      ]
    },
    "visits": {
      "outlier_thresholds": {
        "lower": 1,
        "upper": 7000
      },
      "actions": [
        "Identify and review high visit counts to determine if they reflect genuine traffic spikes.",
        "Review entries with low visits (1) to ensure they are valid, particularly in context."
      ]
    },
    "productClick": {
      "outlier_thresholds": {
        "lower": 0,
        "upper": 32460
      },
      "actions": [
        "Remove or review high product click counts exceeding 15000, as these may be influenced by bot activity.",
        "Check records with zero clicks for legitimacy."
      ]
    },
    "ad

In [9]:
transaction_report = analysis_results["transaction_analysis"]["transaction_report"]
print(transaction_report)

{
  "riskAssessment": {
    "summary": {
      "totalTransactions": 52721,
      "meanRevenue": 127554.8,
      "stdRevenue": 336482.4,
      "minRevenue": 0,
      "maxRevenue": 7368215,
      "transactionCountStats": {
        "min": 0,
        "25thPercentile": 0,
        "50thPercentile": 2,
        "75thPercentile": 11,
        "max": 1113
      }
    },
    "suspiciousIndicators": {
      "highValueTransactions": {
        "count": 2636,
        "examples": [
          {
            "date": "2020-05-11",
            "source": "facebook",
            "revenue": 1289066,
            "adSpend": "₱817,514"
          },
          {
            "date": "2020-03-26",
            "source": "google",
            "revenue": 1406743,
            "adSpend": "₱357,920"
          },
          {
            "date": "2020-07-09",
            "source": "(direct)",
            "revenue": 1712012,
            "adSpend": "₱881,828"
          }
        ]
      },
      "inconsistentDeviceUsage": {
  

## Data Processing

### **Data Processing Agent:**

The **Data Processing Agent** orchestrates a series of operations, including **data type conversions**, **missing value handling**, **outlier detection**, and **anomaly filtering**, using recommendations to guide each step. By integrating these diverse agents, the Data Processing Agent ensures that data quality is maintained, outliers are addressed, and any anomalies are identified and removed, resulting in a clean, reliable dataset ready for use.

1. **Data Type Conversion Agent**: This agent ensures the correct data types for each column, transforming data based on specific recommendations, such as converting date fields to datetime, categorical data to categories, and numerical data to the appropriate type (e.g., float64). It ensures data integrity during type conversion.

2. **Missing Values Agent**: This agent handles missing or null values by filling them with the appropriate statistical measure, such as the **median** for normally distributed columns or the **mode** for skewed distributions. It ensures that missing data does not negatively impact the dataset’s usability.

3. **Outlier Agent**: This agent identifies outliers in the data using statistical methods like the **Z-score method** and either removes or normalizes extreme values that fall outside acceptable thresholds. It prevents outliers from skewing the analysis or models that will be built on the data.

4. **Anomaly Agent**: The anomaly agent detects and filters out anomalies based on **quantile-based filtering** techniques. It uses the 1st and 99th percentiles to remove extreme values that may distort the data, ensuring that the dataset remains representative of typical patterns.


In [17]:
import pandas as pd
import numpy as np
from scipy import stats
from swarm import Swarm, Agent
import logging

class DataPreprocessing:
    def __init__(self, cleaning_report, anomaly_report):
        """
        Initialize the data preprocessing swarm with cleaning and anomaly reports

        Args:
            cleaning_report (dict): Detailed cleaning recommendations
            anomaly_report (dict): Anomaly detection recommendations
        """
        self.cleaning_report = cleaning_report
        self.anomaly_report = anomaly_report
        self.client = Swarm()
        self._setup_agents()

    def _setup_agents(self):
        """
        Set up specialized agents for different preprocessing tasks
        """
        # Data Type Conversion Agent
        def convert_data_types(dataframe):
            return self._convert_data_types(dataframe)

        self.type_conversion_agent = Agent(
            name="DataTypeAgent",
            model="gpt-4o-mini",
            instructions="Convert data types based on provided recommendations. Handle type conversions carefully, ensuring data integrity.",
            functions=[convert_data_types]
        )

        # Missing Value Handling Agent
        def handle_missing_values(dataframe):
            return self._handle_missing_values(dataframe)

        self.missing_values_agent = Agent(
            name="MissingValuesAgent",
            instructions="Handle missing values using appropriate statistical techniques. Prefer median for normal distributions, mode for skewed distributions.",
            functions=[handle_missing_values]
        )

        # Outlier Handling Agent
        def process_outliers(dataframe):
            return self._process_outliers(dataframe)

        self.outlier_agent = Agent(
            name="OutlierAgent",
            instructions="Detect and handle outliers using Z-score method. Remove or normalize outliers based on pre-defined thresholds.",
            functions=[process_outliers]
        )

        # Anomaly Filtering Agent
        def filter_anomalies(dataframe):
            return self._filter_anomalies(dataframe)

        self.anomaly_agent = Agent(
            name="AnomalyAgent",
            instructions="Identify and filter out data anomalies using quantile-based filtering. Ensure data quality while minimizing information loss.",
            functions=[filter_anomalies]
        )

    def preprocess_data(self, dataframe):
        """
        Orchestrate preprocessing steps using agent-based workflow

        Args:
            dataframe (pd.DataFrame): Input dataframe to be preprocessed

        Returns:
            pd.DataFrame: Fully preprocessed dataframe
        """
        # Validate input
        if dataframe is None or dataframe.empty:
            logging.warning("Input dataframe is None or empty")
            return pd.DataFrame()

        # Preprocessing pipeline with agent handoffs
        preprocessing_steps = [
            (self.type_conversion_agent, "Convert data types"),
            (self.missing_values_agent, "Handle missing values"),
            (self.outlier_agent, "Process outliers"),
            (self.anomaly_agent, "Filter anomalies")
        ]

        dataframe['revenue'] = df['revenue'].apply(lambda x: int(x.replace("₱", "").replace(",", "")))
        dataframe['ad spend'] = df['ad spend'].apply(lambda x: int(x.replace("₱", "").replace(",", "")))
        processed_df = dataframe.copy().to_dict()
        for agent, step_name in preprocessing_steps:
            try:
                response = self.client.run(
                    agent=agent,
                    messages=[{"role": "user", "content": step_name, "dataframe": processed_df}]
                )
                processed_df = response.messages[-1].get("dataframe", processed_df)
            except Exception as e:
                logging.error(f"Error in {step_name}: {e}")
                continue

        return processed_df

    def _convert_data_types(self, dataframe_dict):
        """
        Convert data types based on cleaning recommendations

        Returns:
            pd.DataFrame: Dataframe with corrected data types
        """
        df = pd.DataFrame.from_dict(dataframe_dict)
        recommendations = self.cleaning_report.get('recommendations', {}).get('data_type_issues', {})

        for column, type_info in recommendations.items():
            if column in df.columns:
                current_type = type_info.get('current_type')
                recommended_type = type_info.get('recommended_type')
                try:
                    if recommended_type == 'datetime':
                        df[column] = pd.to_datetime(df[column], errors='coerce')
                    elif recommended_type == 'categorical':
                        df[column] = pd.Categorical(df[column])
                    elif recommended_type == 'boolean':
                        df[column] = df[column].map({'yes': True, 'no': False})
                    elif recommended_type == 'float64':
                        df[column] = pd.to_numeric(df[column].replace('[\$,]', '', regex=True), errors='coerce')
                except Exception as e:
                    logging.error(f"Error converting {column} to {recommended_type}: {e}")

        return df

    def _handle_missing_values(self, dataframe_dict):
        """
        Handle missing values using appropriate statistical techniques


        Returns:
            pd.DataFrame: Dataframe with handled missing values
        """
        df = pd.DataFrame.from_dict(dataframe_dict)
        missing_config = self.cleaning_report.get('recommendations', {}).get('missing_values', {})

        if missing_config.get('action') == 'None required':
            return df

        for column in df.columns:
            if df[column].isnull().any():
                try:
                    if self._is_normally_distributed(df[column]):
                        df[column].fillna(df[column].median(), inplace=True)
                    else:
                        df[column].fillna(df[column].mode()[0], inplace=True)
                except Exception as e:
                    logging.error(f"Error handling missing values in {column}: {e}")

        return df

    def _process_outliers(self, dataframe_dict):
        """
        Process outliers using Z-score method

        Returns:
            pd.DataFrame: Dataframe with processed outliers
        """
        df = pd.DataFrame.from_dict(dataframe_dict)
        outlier_config = self.anomaly_report.get('cleaning_recommendations', {})

        for column, config in outlier_config.items():
            if column in df.columns and 'outliers' in config:
                try:
                    outliers_info = config['outliers']
                    if outliers_info.get('remove'):
                        threshold = outliers_info['remove'].get('values_above')
                        df = df[df[column] <= threshold]
                except Exception as e:
                    logging.error(f"Error processing outliers in {column}: {e}")

        return df

    def _filter_anomalies(self, dataframe):
        """
        Filter anomalies using quantile-based approach

        Args:
            dataframe (pd.DataFrame): Input dataframe

        Returns:
            pd.DataFrame: Filtered dataframe
        """
        df = dataframe.copy()
        anomaly_config = self.anomaly_report.get('cleaning_recommendations', {})

        for column, config in anomaly_config.items():
            if column in df.columns:
                try:
                    # Filter using 1st and 99th percentiles if normalize is true
                    if config.get('outliers', {}).get('normalize', False):
                        upper_bound = df[column].quantile(0.99)
                        lower_bound = df[column].quantile(0.01)
                        df = df[(df[column] <= upper_bound) & (df[column] >= lower_bound)]
                except Exception as e:
                    logging.error(f"Error filtering anomalies in {column}: {e}")

        return df

    def _is_normally_distributed(self, series, alpha=0.05):
        """
        Check if a series is normally distributed

        Args:
            series (pd.Series): Input series to test
            alpha (float): Significance level for the test

        Returns:
            bool: True if normally distributed, False otherwise
        """
        try:
            cleaned_series = series.dropna()
            if len(cleaned_series) < 3:
                return False

            _, p_value = stats.shapiro(cleaned_series)
            return p_value > alpha
        except Exception as e:
            logging.error(f"Distribution test error: {e}")
            return False


In [18]:
preprocessing_agent = DataPreprocessing(cleaning_report, anomaly_report)

In [19]:
processed_df = preprocessing_agent.preprocess_data(df)

ERROR:root:Error in Convert data types: DataFrame constructor not properly called!


In [20]:
processed_df = pd.DataFrame(processed_df)
processed_df.head(5)

,date,source,medium,delivery_available,device_type,promo_activated,filter_used,pageviews,visits,productClick,addToCart,checkout,transactions,revenue,ad spend
0,2020-05-11,google,organic,no data,PC,no,no,4087,1233,5240,1048,672.00,90,456877,384039
1,2020-05-11,facebook,cpc,no data,mobile,yes,no,4326,544,9930,1984,1812.48,217,1289066,817514
2,2020-05-11,google,cpc,no data,mobile,no,no,3891,1450,5460,1090,766.72,100,554427,435105
3,2020-05-11,google,cpc,no data,PC,no,no,2456,854,4250,848,520.96,71,416561,635599
4,2020-05-11,facebook,organic,no data,PC,no,no,2828,1000,4110,824,449.28,62,326176,428962


## Marketing Intelligent

The **Market Trend Analysis Agent** is a specialized agent designed to analyze market data and identify key trends, shifts in consumer behavior, and emerging opportunities. By processing data on traffic sources, device performance, and revenue patterns, this agent delivers valuable insights that help businesses understand current market dynamics. It performs detailed analysis such as traffic breakdown by source, conversion rates, and device-specific revenue performance. Based on this data, the agent identifies trends like the growing dominance of mobile traffic and the importance of paid media. After completing the analysis, the **MarketTrendAnalysisAgent** transfers its findings to the **MarketRecommendationAgent** to generate actionable recommendations. The output is delivered in a concise JSON format, summarizing key trends, consumer behavior shifts, and opportunities.

The **Market Recommendation Agent** takes the results from the **Market Trend Analysis Agent** and generates strategic, actionable recommendations based on market trends. It synthesizes data-driven insights to identify business priorities and specific action items that businesses can implement to drive growth. The recommendations include key strategic initiatives, such as enhancing mobile marketing capabilities and optimizing paid media channels. It also provides specific action items, such as developing mobile-responsive landing pages or improving audience targeting for paid media. Additionally, the **Market Recommendation Agent** quantifies the potential impact on revenue growth and conversion rates while highlighting risk mitigation strategies, ensuring that businesses can prioritize efforts and manage potential challenges effectively.


In [21]:
import os
import pandas as pd
from swarm import Swarm, Agent
from typing import Dict, Any, List

class MarketTrendAnalysisAgent(Agent):
    def __init__(self):
        def transfer_to_recommendation_agent():
            return recommendation_agent

        super().__init__(
            name="MarketTrendAnalysisAgent",
            instructions="""
            You are a market trend analysis specialist.
            Your task is to analyze market data and identify key trends,
            consumer behavior shifts, and emerging opportunities.
            Provide a comprehensive but concise analysis.

            Respond ONLY with a JSON format output directly addressing the analysis in market trends. No json in the start of the string.
            """,
            functions=[transfer_to_recommendation_agent]
        )

    def analyze_market_trends(self, df: pd.DataFrame) -> Dict[str, Any]:
        """
        Perform market trend analysis

        :param df: Input pandas DataFrame
        :return: Detailed market trend analysis
        """
        # Preprocess data
        traffic_breakdown = df.groupby('source')['transactions'].agg([
            'count',
            'mean',
            ('conversion_rate', lambda x: x.sum() / len(df) * 100)
        ]).reset_index()

        device_performance = df.groupby('device_type')['revenue'].agg([
            'mean',
            'sum',
            'count'
        ]).reset_index()

        # Construct analysis summary
        analysis = {
            "key_trends": [
                "Mobile traffic shows highest conversion rates",
                "Paid media drives significant revenue"
            ],
            "consumer_behavior_shifts": [
                "Increasing mobile device usage",
                "Growing preference for targeted advertising"
            ],
            "emerging_opportunities": [
                "Mobile-first marketing strategies",
                "Personalized ad targeting"
            ],
            "data_summary": {
                "traffic_breakdown": traffic_breakdown.to_dict(orient='records'),
                "device_performance": device_performance.to_dict(orient='records')
            }
        }

        return analysis

class MarketRecommendationAgent(Agent):
    def __init__(self):
        super().__init__(
            name="MarketRecommendationAgent",
            instructions="""
            You are a strategic recommendations specialist.
            Based on market trend analysis, generate actionable
            strategic recommendations for business growth.

            Provide clear, data-driven insights and prioritized action items.
            """
        )

    def generate_recommendations(self, trend_analysis: Dict[str, Any]) -> Dict[str, Any]:
        """
        Generate strategic recommendations

        :param trend_analysis: Market trend analysis results
        :return: Strategic recommendations
        """
        recommendations = {
            "strategic_priorities": [
                "Enhance mobile marketing capabilities",
                "Optimize paid media channels"
            ],
            "action_items": {
                "mobile_strategy": [
                    "Develop mobile-responsive landing pages",
                    "Create mobile-specific ad creatives"
                ],
                "paid_media_optimization": [
                    "Implement advanced audience targeting",
                    "Increase budget for high-performing channels"
                ]
            },
            "potential_impact": {
                "revenue_growth": 0.15,
                "conversion_rate_improvement": 0.10
            },
            "risk_mitigation": {
                "channel_diversification": "Reduce dependency on single traffic source",
                "continuous_monitoring": "Regular performance reviews"
            }
        }

        return recommendations


In [22]:
# Initialize Swarm
client = Swarm()

# Create recommendation agent first (to allow transfer)
recommendation_agent = MarketRecommendationAgent()

# Create trend analysis agent with ability to transfer
trend_analysis_agent = MarketTrendAnalysisAgent()

### Market Trend Analysis

In [23]:
# Run initial workflow with trend analysis agent
response = client.run(
    agent=trend_analysis_agent,
    messages=[
        {"role": "user", "content": "Analyze market trends for our digital marketing data"},
        {"role": "system", "content": str(processed_df)}
    ]
)

# Print trend analysis
trend_analysis = response.messages[-1]["content"]
print(trend_analysis)

{
  "key_trends": {
    "device_usage": {
      "mobile": {
        "dominant_in_source": ["facebook", "google"],
        "performance_metrics": {
          "higher_visits": true,
          "high_productClick": true
        }
      },
      "PC": {
        "dominant_in_source": ["google", "direct"],
        "performance_metrics": {
          "less_visits": true,
          "good_productClick": false
        }
      }
    },
    "source_performance": {
      "google_cpc": {
        "high_revenue_generation": true,
        "high_addToCart_rate": true
      },
      "facebook_cpc": {
        "good_revenue_with_less_spend": true,
        "promo_influence": "yes"
      }
    }
  },
  "consumer_behavior_shifts": {
    "promo_effectiveness": {
      "positive_influence_on_facebook": true,
      "less_effect_on_emails": true
    },
    "filter_usage": {
      "enhanced_with_device_mobile": true,
      "productClick_increase": true
    }
  },
  "emerging_opportunities": {
    "cross_device_targe

### Strategic Marketing Recommendations

In [24]:
# Transfer to recommendation agent
recommendation_response = client.run(
    agent=recommendation_agent,
    messages=[
        {"role": "user", "content": "Generate strategic recommendations based on the trend analysis"},
        {"role": "system", "content": trend_analysis}
    ]
)

# Print recommendations
strategic_recommendations = recommendation_response.messages[-1]["content"]
print(strategic_recommendations)

Based on the analyzed market trends, the following strategic recommendations are prioritized to drive business growth:

1. **Optimize Mobile Device Engagement:**
   - **Action Items:**
     - Enhance mobile-specific marketing strategies, focusing on platforms like Facebook and Google, where mobile usage is dominant.
     - Implement mobile-first design for a seamless user experience, emphasizing product discovery and click-through actions.
     - Consider personalized notifications and reminders leveraging mobile device capabilities to boost conversions.

2. **Leverage Google CPC for Revenue Growth:**
   - **Action Items:**
     - Increase investment in Google CPC campaigns, focusing on high-conversion keywords and ad formats.
     - Develop targeted campaigns that capitalize on Google's high Add-to-Cart rates, ensuring a streamlined checkout process to minimize cart abandonment.
     - Regularly analyze Google CPC performance to optimize bidding strategies and enhance ROI.

3. **Enhan

## Customer Experience


The **User Segmentation Agent** is an agent designed to segment users based on marketplace data, enabling targeted marketing and personalized customer strategies. This agent performs advanced user clustering using machine learning techniques, specifically K-Means clustering, to categorize users into distinct segments. It preprocesses user data by calculating key metrics such as conversion rates, revenue per visit, and engagement scores, then applies normalization to ensure accuracy in the clustering process. The agent analyzes user behaviors, such as visits, pageviews, transactions, and device usage, to provide detailed insights into each segment. Once the segmentation process is complete, the **User Segmentation Agent** prepares the data for the **Personalization Agent**, enabling businesses to tailor their strategies to each user group.


The **Personalization Agent** is responsible for translating user segmentation data into actionable, personalized strategies to enhance the customer experience. Based on insights from the **User Segmentation Agent**, it creates detailed personalization plans, offering tailored marketing and communication strategies for each user segment. This agent crafts user personas, defines communication strategies (including the preferred channels and messaging tone), and recommends targeted marketing tactics like personalized mobile campaigns and product recommendations. The **Personalization Agent** helps organizations improve conversion rates by implementing strategies such as segment-specific landing pages and promotions. Once the personalization strategies are ready, the agent prepares the data for the **Reporting Agent** to generate a comprehensive report.

The **Reporting Agent** consolidates and compiles detailed customer experience reports by combining insights from the **User Segmentation Agent** and the **Personalization Agent**. It generates comprehensive, markdown-formatted reports that summarize user segment statistics and corresponding personalization strategies. This agent’s role is to synthesize the segmented data and the personalized strategies into clear, actionable, and strategic reports that are easy to interpret. These reports can be used by decision-makers to understand user behaviors and the impact of personalized strategies, enabling businesses to refine their approach to customer experience and marketing.


In [25]:
import os
import pandas as pd
import numpy as np
from typing import Dict, List, Any
from swarm import Swarm, Agent

# Data Processing
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

class UserSegmentationAgent(Agent):
    def __init__(self):
        def transfer_to_personalization_agent():
            return personalization_agent

        super().__init__(
            name="UserSegmentationAgent",
            instructions="""
            You are a user segmentation specialist responsible for:
            1. Preprocessing marketplace data
            2. Performing advanced user clustering
            3. Generating detailed user segment insights
            4. Preparing data for personalization strategy

            Respond ONLY with a JSON format output directly addressing the segmented data. No json in the start of the string.
            """,
            functions=[transfer_to_personalization_agent]
        )

    def segment_users(self, n_clusters: int = 5, preprocessed_df = pd.DataFrame) -> Dict:
        """
        Perform advanced user segmentation

        Args:
            n_clusters (int): Number of user segments

        Returns:
            Detailed user segment information
        """
        if processed_df is None:
            raise ValueError("No data has been set. Use set_data() method first.")

        df = processed_df.copy()

        # Feature Engineering
        df['conversion_rate'] = df['transactions'] / df['visits']
        df['revenue_per_visit'] = df['revenue'] / df['visits']
        df['engagement_score'] = (df['pageviews'] / df['visits']) * df['conversion_rate']

        # Select clustering features
        clustering_features = [
            'visits', 'pageviews', 'transactions',
            'conversion_rate', 'revenue_per_visit'
        ]

        # Normalize features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(self.preprocessed_data[clustering_features])

        # Perform K-Means Clustering
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        self.preprocessed_data['user_segment'] = kmeans.fit_predict(X_scaled)

        # Analyze Segments
        segments = {}
        for segment in range(n_clusters):
            segment_data = self.preprocessed_data[self.preprocessed_data['user_segment'] == segment]

            segments[segment] = {
                'size': len(segment_data),
                'avg_visits': segment_data['visits'].mean(),
                'avg_conversion_rate': segment_data['conversion_rate'].mean(),
                'avg_revenue_per_visit': segment_data['revenue_per_visit'].mean(),
                'source': segment_data['source'].value_counts().to_dict(),
                'device_types': segment_data['device_type'].value_counts().to_dict()
            }

        self.user_segments = segments
        return segments

class PersonalizationAgent(Agent):
    def __init__(self):
        def transfer_to_reporting_agent():
            return reporting_agent

        super().__init__(
            name="PersonalizationAgent",
            instructions="""
            You are a customer experience personalization strategist.
            Your role is to:
            1. Develop personalized strategies for each user segment
            2. Create targeted communication and marketing approaches
            3. Generate actionable insights for improving customer experience

            Respond ONLY with a JSON format output directly addressing the personalization strategy. No json in the start of the string.
            """,
            functions=[transfer_to_reporting_agent]
        )

    def generate_personalization_strategy(self, segment_data: Dict) -> Dict:
        """
        Generate personalized strategy for a user segment

        Args:
            segment_data (Dict): Segment insights

        Returns:
            Personalization strategy details
        """
        personalization_strategy = {
            "user_persona": {
                "profile": f"Tech-savvy {segment_data['size']} user segment",
                "key_characteristics": {
                    "avg_visits": segment_data['avg_visits'],
                    "conversion_rate": segment_data['avg_conversion_rate'],
                    "revenue_potential": segment_data['avg_revenue_per_visit']
                }
            },
            "communication_strategy": {
                "primary_channels": list(segment_data['source'].keys()),
                "messaging_tone": "Informative and tech-focused"
            },
            "marketing_recommendations": {
                "top_devices": list(segment_data['device_types'].keys()),
                "targeted_campaigns": "Personalized mobile marketing"
            },
            "conversion_tactics": [
                "Targeted mobile landing pages",
                "Personalized product recommendations",
                "Segment-specific promotional offers"
            ]
        }

        return personalization_strategy

class ReportingAgent(Agent):
    def __init__(self):
        super().__init__(
            name="ReportingAgent",
            instructions="""
            You are a reporting and insights specialist.
            Compile comprehensive customer experience reports
            that synthesize segmentation and personalization insights.

            Generate clear, actionable, and strategic reports.
            """
        )

    def generate_comprehensive_report(self, segments: Dict, personalization_strategies: Dict) -> str:
        """
        Generate a comprehensive customer experience report

        Args:
            segments (Dict): User segment insights
            personalization_strategies (Dict): Personalization strategies

        Returns:
            Markdown-formatted report
        """
        report = "# Customer Experience Intelligence Report\n\n"

        for segment_id, segment_data in segments.items():
            report += f"## Segment {segment_id} Analysis\n\n"

            # Segment Statistics
            report += "### Segment Statistics\n"
            for key, value in segment_data.items():
                report += f"- **{key.replace('_', ' ').title()}**: {value}\n"

            # Personalization Strategy
            strategy = personalization_strategies.get(segment_id, {})
            report += "\n### Personalization Strategy\n"
            report += f"```json\n{strategy}\n```\n\n"

        return report


In [26]:
# Initialize Swarm
client = Swarm()

# Create agents (reporting agent first to allow transfer)
reporting_agent = ReportingAgent()
personalization_agent = PersonalizationAgent()

In [29]:
# Create segmentation agent with transfer capability
segmentation_agent = UserSegmentationAgent()

# Run Segmentation Workflow
segmentation_response = client.run(
    agent=segmentation_agent,
    messages=[
        {"role": "user", "content": "Perform advanced user segmentation"},
        {"role": "system", "content": str(processed_df)}
    ]
)

# Extract segments from response
segments = segmentation_response.messages[-1]["content"]

print(segments)

{
  "UserSegments": [
    {
      "SegmentName": "High Value Mobile Users",
      "Criteria": {
        "device_type": "mobile",
        "transactions": ">= 100",
        "promo_activated": "yes"
      },
      "Insights": {
        "Description": "Users primarily accessing the platform through mobile devices, engaging heavily in transactions, and responsive to promotions.",
        "Potential": "High potential for upselling and engagement through mobile-centric promotions."
      },
      "Strategies": [
        "Develop mobile-first promotional campaigns.",
        "Enhance mobile UX and interfaces specifically for transaction ease.",
        "Provide exclusive mobile-based offers."
      ]
    },
    {
      "SegmentName": "Organic Desktop Users",
      "Criteria": {
        "medium": "organic",
        "device_type": "PC",
        "transactions": "< 50"
      },
      "Insights": {
        "Description": "Users visiting the platform organically through desktop but with low transact

In [30]:
# Run Personalization Workflow
personalization_response = client.run(
    agent=personalization_agent,
    messages=[
        {"role": "user", "content": "Generate personalization strategies for user segments"},
        {"role": "system", "content": str(segments)}
    ]
)

# Extract personalization strategies
personalization_strategies = personalization_response.messages[-1]["content"]
print(personalization_strategies)


{
  "UserSegments": [
    {
      "SegmentName": "High Value Mobile Users",
      "Criteria": {
        "device_type": "mobile",
        "transactions": ">= 100",
        "promo_activated": "yes"
      },
      "Insights": {
        "Description": "Users primarily accessing the platform through mobile devices, engaging heavily in transactions, and responsive to promotions.",
        "Potential": "High potential for upselling and engagement through mobile-centric promotions."
      },
      "Strategies": [
        "Develop mobile-first promotional campaigns.",
        "Enhance mobile UX and interfaces specifically for transaction ease.",
        "Provide exclusive mobile-based offers."
      ]
    },
    {
      "SegmentName": "Organic Desktop Users",
      "Criteria": {
        "medium": "organic",
        "device_type": "PC",
        "transactions": "< 50"
      },
      "Insights": {
        "Description": "Users visiting the platform organically through desktop but with low transact

In [33]:
# Generate Comprehensive Report
report_response = client.run(
    agent=reporting_agent,
    messages=[
        {"role": "user", "content": "Generate comprehensive customer experience report"},
        {"role": "system", "content": f"Segments: {segments}, Strategies: {personalization_strategies}"}
    ]
)

# Print Final Report
customer_experience_report = report_response.messages[-1]["content"]
print(customer_experience_report)

# Comprehensive Customer Experience Report

## Overview

This report provides insights into the customer segments of our platform and identifies potential strategies to enhance customer experience and engagement. The segments are categorized based on user behavior, device usage, and promotional interactions, aiming to leverage their unique characteristics for personalized marketing and customer satisfaction improvements.

## Customer Segments

### 1. High Value Mobile Users
- **Criteria**: Utilizes mobile devices, conducts 100 or more transactions, participates in promotions.
- **Insights**: This group makes significant use of promotions and mobile transactions, indicating a preference for mobility and digital offers.
- **Potential**: There is a high potential to increase revenue and engagement via targeted mobile experiences.

#### Strategies:
1. Develop exclusive promotions for mobile users.
2. Optimize mobile interfaces for seamless transaction experiences.
3. Offer personalized, mo

## Visual Recommender

The **Data Visualizer Agent** recommends the most appropriate graphs based on a dataset's columns and the context of an accompanying report. By analyzing the data types, relationships between variables, and the insights described in the report, it suggests the most effective visualizations, such as bar charts, scatter plots, or line graphs, tailored to highlight key trends and comparisons. The agent offers clear explanations on why a specific visualization is chosen, helping users easily interpret and communicate their data insights.

In [42]:
import os
import pandas as pd
import numpy as np
from typing import Dict, List, Any
from swarm import Swarm, Agent

class DataVisualizerAgent(Agent):
    def __init__(self):
        super().__init__(
            name="DataVisualizerAgent",
            instructions="""
              You are a data visualization expert. Analyze the following report data
              and recommend the most suitable visualization graphs. For each recommendation,
              provide:
              - Graph type (bar chart, scatter plot, etc.)
              - X-axis column
              - Y-axis column
              - Columns needed (only consider the columns that is in the dataset columns)
              - Brief description of why this visualization is appropriate

              Respond strictly in the following JSON format. No json in the start of the string.:
              {{
                  "recommended_graphs": [
                      {{
                          "type": "string",
                          "x_axis": "string",
                          "y_axis": "string",
                          "columns_needed": ["string"],
                          "title": "string",
                          "description": "string"
                      }}
                  ]
              }}
              """
        )


In [80]:
# Create visualizer agent with transfer capability
visualizer_agent = DataVisualizerAgent()
column_str = ', '.join(processed_df.columns)

# Run Visualization Workflow
visualization_response = client.run(
    agent=visualizer_agent,
    messages=[
        {"role": "user", "content": "Recommend a visualozations for the report and given dataset columns."},
        {"role": "system", "content": f""" You are a data visualization expert.
                Report Data: {trend_analysis}
                Dataset Columns: {column_str}"""}
    ]
)

In [81]:
# Extract recommendations from response
visual_recommendations = visualization_response.messages[-1]["content"]
print(visual_recommendations)

{
    "recommended_graphs": [
        {
            "type": "bar chart",
            "x_axis": "source",
            "y_axis": "revenue",
            "columns_needed": ["source", "revenue"],
            "title": "Revenue by Source",
            "description": "A bar chart is suitable for comparing the revenue generated from different sources such as Google CPC and Facebook CPC. This helps identify which sources are most profitable."
        },
        {
            "type": "scatter plot",
            "x_axis": "ad spend",
            "y_axis": "revenue",
            "columns_needed": ["ad spend", "revenue"],
            "title": "Ad Spend vs Revenue",
            "description": "A scatter plot is appropriate for analyzing the relationship between ad spend and revenue generated. This visualization can help determine the efficiency and effectiveness of ad spending on each platform."
        },
        {
            "type": "stacked bar chart",
            "x_axis": "device_type",
       

In [82]:
visual_recommendations = json.loads(visual_recommendations)

In [84]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go

class PlotlyVisualization:
    def __init__(self, dataset, recommended_graphs):
        """
        Initialize the visualization agent with dataset and graph recommendations.

        :param dataset: pandas DataFrame containing the data
        :param recommended_graphs: List of graph specifications
        """
        self.dataset = dataset
        self.recommended_graphs = recommended_graphs

    def validate_columns(self, graph_spec):
        """
        Validate that required columns exist in the dataset.

        :param graph_spec: Dictionary containing graph specification
        :return: Boolean indicating column availability
        """
        columns_needed = graph_spec.get('columns_needed', [])
        return all(col in self.dataset.columns for col in columns_needed)

    def generate_visualizations(self):
        """
        Generate all recommended visualizations.

        :return: Dictionary of graph specifications and their corresponding Plotly figures
        """
        visualizations = {}

        for graph_spec in self.recommended_graphs:
            # Validate columns before attempting to create visualization
            if not self.validate_columns(graph_spec):
                print(f"Skipping {graph_spec['title']} - Required columns not found")
                continue

            # Create visualization based on graph type
            try:
                if graph_spec['type'] == 'bar chart':
                    fig = self._create_bar_chart(graph_spec)
                elif graph_spec['type'] == 'scatter plot':
                    fig = self._create_scatter_plot(graph_spec)
                elif graph_spec['type'] == 'line chart':
                    fig = self._create_line_chart(graph_spec)
                elif graph_spec['type'] == 'box plot':
                    fig = self._create_boxplot(graph_spec)
                elif graph_spec['type'] == 'histogram':
                    fig = self._create_histogram(graph_spec)
                else:
                    print(f"Unsupported graph type: {graph_spec['type']}")
                    continue

                visualizations[graph_spec['title']] = {
                    'figure': fig,
                    'description': graph_spec.get('description', '')
                }
            except Exception as e:
                print(f"Error creating {graph_spec['title']}: {str(e)}")


        return visualizations

    def _create_bar_chart(self, graph_spec):
        """
        Create a bar chart based on graph specification.

        :param graph_spec: Dictionary containing bar chart specification
        :return: Plotly Figure object
        """
        # Support for color on x-axis
        color = graph_spec['x_axis']

        if color:
            fig = px.bar(
                self.dataset,
                x=graph_spec['x_axis'],
                y=graph_spec['y_axis'],
                color=color,
                title=graph_spec['title']
            )
        else:
            fig = px.bar(
                self.dataset,
                x=graph_spec['x_axis'],
                y=graph_spec['y_axis'],
                title=graph_spec['title']
            )

        fig.update_layout(
            xaxis_title=graph_spec.get('x_axis_title', graph_spec['x_axis']),
            yaxis_title=graph_spec.get('y_axis_title', graph_spec['y_axis'])
        )
        return fig

    def _create_scatter_plot(self, graph_spec):
        """
        Create a scatter plot based on graph specification.

        :param graph_spec: Dictionary containing scatter plot specification
        :return: Plotly Figure object
        """
        # Support for color on x-axis and additional size parameter
        color = graph_spec['x_axis']

        if color:
            fig = px.scatter(
                self.dataset,
                x=graph_spec['x_axis'],
                y=graph_spec['y_axis'],
                color=color,
                title=graph_spec['title']
            )
        else:
            fig = px.scatter(
                self.dataset,
                x=graph_spec['x_axis'],
                y=graph_spec['y_axis'],
                title=graph_spec['title']
            )

        fig.update_layout(
            xaxis_title=graph_spec.get('x_axis_title', graph_spec['x_axis']),
            yaxis_title=graph_spec.get('y_axis_title', graph_spec['y_axis'])
        )
        return fig

    def _create_line_chart(self, graph_spec):
        """
        Create a line chart based on graph specification.

        :param graph_spec: Dictionary containing line chart specification
        :return: Plotly Figure object
        """
        # Support for color on x-axis
        color = graph_spec['x_axis']

        if color:
            fig = px.line(
                self.dataset,
                x=graph_spec['x_axis'],
                y=graph_spec['y_axis'],
                color=color,
                title=graph_spec['title']
            )
        else:
            fig = px.line(
                self.dataset,
                x=graph_spec['x_axis'],
                y=graph_spec['y_axis'],
                title=graph_spec['title']
            )

        fig.update_layout(
            xaxis_title=graph_spec.get('x_axis_title', graph_spec['x_axis']),
            yaxis_title=graph_spec.get('y_axis_title', graph_spec['y_axis'])
        )
        return fig

    def _create_boxplot(self, graph_spec):
        """
        Create a boxplot based on graph specification.

        :param graph_spec: Dictionary containing boxplot specification
        :return: Plotly Figure object
        """
        # Support for color grouping
        color = graph_spec.get('color')

        if color:
            fig = px.box(
                self.dataset,
                x=graph_spec.get('x_axis'),
                y=graph_spec['y_axis'],
                color=color,
                title=graph_spec['title']
            )
        else:
            fig = px.box(
                self.dataset,
                x=graph_spec.get('x_axis'),
                y=graph_spec['y_axis'],
                title=graph_spec['title']
            )

        fig.update_layout(
            xaxis_title=graph_spec.get('x_axis_title', graph_spec.get('x_axis', '')),
            yaxis_title=graph_spec.get('y_axis_title', graph_spec['y_axis'])
        )
        return fig

    def _create_histogram(self, graph_spec):
        """
        Create a histogram based on graph specification.

        :param graph_spec: Dictionary containing histogram specification
        :return: Plotly Figure object
        """
        # Support for color grouping and additional parameters
        color = graph_spec.get('color')

        if color:
            fig = px.histogram(
                self.dataset,
                x=graph_spec['x_axis'],
                color=color,
                title=graph_spec['title'],
                marginal=graph_spec.get('marginal'),  # Optional: 'rug', 'box', 'violin'
                nbins=graph_spec.get('nbins')  # Optional: number of bins
            )
        else:
            fig = px.histogram(
                self.dataset,
                x=graph_spec['x_axis'],
                title=graph_spec['title'],
                marginal=graph_spec.get('marginal'),
                nbins=graph_spec.get('nbins')
            )

        fig.update_layout(
            xaxis_title=graph_spec.get('x_axis_title', graph_spec['x_axis']),
            yaxis_title=graph_spec.get('y_axis_title', 'Count')
        )
        return fig


def create_visualizations(dataset, recommended_graphs):
    """
    Convenience function to create visualizations from a dataset and graph specifications.

    :param dataset: pandas DataFrame
    :param recommended_graphs: List of graph specifications
    :return: Dictionary of generated visualizations
    """
    agent = PlotlyVisualization(dataset, recommended_graphs)
    visualizations = agent.generate_visualizations()
    return visualizations

In [85]:
# Generate visualizations
visualizations = create_visualizations(processed_df, visual_recommendations['recommended_graphs'])

Unsupported graph type: stacked bar chart
Unsupported graph type: heatmap


In [86]:
# Show each visualization
for title, visualization_data in visualizations.items():
    visualization_data['figure'].show()  # Display the figure using show()
